# Building a Bitstream

In this step we will be creating a FPGA Bitstream with your HLS core from **[Creating a Vivado HLS Core](2-Creating-A-Vivado-HLS-Core.ipynb)**. We will be using Vivado to create a block diagram, export it as a `.tcl` file, and compiling it into a `.bit` file.

PYNQ uses this `.tcl` file to match FPGA cores to existing drivers. 

To skip this notebook, copy the contents of the `~/PYNQ-HLS/pynqhls/io/` directory to `~/PYNQ-HLS/tutorial/pynqhls/io/` by running the following command on your host computer:

``` bash
    cp ~/PYNQ-HLS/pynqhls/io/* ~/PYNQ-HLS/tutorial/pynqhls/io/
```

This command command can be run in Cygwin, or in a Bash shell in Linux.

## Adding Vivado to your Executable Path

If you have not already, add Vivado to your executable path. In Cygwin, you can do this by running:

``` bash
source C:/Xilinx/Vivado/2017.1/settings64.sh
```

Or on Linux: 

``` bash
source /opt/Xilinx/Vivado/2017.1/settings64.sh
```

## Creating a Vivado Project

We will begin by creating a Vivado project. On your host computer, navigate to the `tutorial/pynqhls/io` folder of the PYNQ-HLS repository using your terminal:

```bash
    cd ~/PYNQ-HLS/tutorial/pynqhls/io
```

In this directory we have provided a makefile that will: 

1. Create a Vivado project for the PYNQ board
2. Add the `ctrlloop` IP path to the Vivado project
3. Create a block diagram using `io.tcl`
4. Instantiate a correctly-configured ZYNQ ARM Processing System 
5. Compile the project into a FPGA bitstream

To run the makefile, run the following command from from the directory above.

``` bash
    make
```

This will perform the steps above. When this is complete run the following command: 

```bash
    vivado io/io.xpr
```

You should now see the following window: 


<img src="pictures/vivado_io_splash.png" alt="IO Project in Vivado" style="width: 768px;"/>

# Adding IP to the Block Diagram

Once the project has been created, we must add the HLS core, a DMA engine, and communication cores. 

Open the block diagram by expanding the top level and double clicking on `io_i` highlighted in the figure above. You will see the window shown below: 

<img src="pictures/vivado_io_bd.png" alt="Initial Block Diagram" style="width: 768px;"/>

Click on the `+` icon to add cores to your block design. In the search box that appears, type `Simple IO`. You should see the following window:

<img src="pictures/vivado_io_core.png" alt="The ctrlloop HLS core" style="width: 256px;"/>

Double click on "Simple IO Core" to add the core to the block diagram. **Change the name to `ctrlLoop` using the properties sidebar.** You should see a block diagram similar to the following: 

<img src="pictures/vivado_io_bd_2.png" alt="The `ctrlloop` HLS core in the IO Block Diagram" style="width: 768px;"/>

We will perform wiring later. Add the following cores to the block diagram, and name them accordingly using the process described above: 

- AXI UART Lite Core (`hlsUartLite`)
    - This is the AXI Core used for UART communication
- AXI Interconnect (`hlsInterconnect`)
    - This routes addressed writes to the AXI UART Controller that originate in the HLS Core
- AXI Interconnect (`plInterconnect`)
    - This routes addressed configuration register writes that originate in the ARM Processing system
- Processor System Reset (`userReset`)
    - This handle the sequencing of resets
- Concat (`plIrqConcat`)
    - This concatenates all IRQ Signals into a single bus.
- AXI Interrupt Controller (`plIntrController`)
    - This is the endpoint for the concatenated bus described above. 
- Concat (`pmodConcat`)
    - This concatenates all IRQ Signals into a single bus.
- Slice (`uartRxSlice`)
    - This removes the UART RX wire from an 8-bit input bus from the PMOD interface through bit-index slicing
- Constant (`xlConstant_0`)
    - This will drive the tri-state buffers for the PMOD interface   


## Configuration

You will need to configure the previous IP Blocks in the following ways: 

### AXI UART Lite Core  (`hlsUartLite`)

First, configure the UART Lite Core. Double click to bring up the window below.

You should match the settings shown in the picture below:
- Baud Rate: 9600
- Data Bits: 8

<img src="pictures/vivado_io_uart.png" alt="Configuring the UART Lite Core" style="width: 768px;"/>

### AXI Interconnect (`hlsInterconnect`)

Next, configure the first AXI Interconnect, named `hlsInterconnect`. 

You should match the settings shown in the picture below:
- Number of Slave Interfaces : 1
- Number of Master Interfaces : 1

<img src="pictures/vivado_io_hlsinterconnect.png" alt="Configuring the HLS Interconnect AXI Switch" style="width: 768px;"/>


### AXI Interconnect (`psInterconnect`)

Next, configure the second AXI Interconnect, named `psInterconnect`. 

You should match the settings shown in the picture below:
- Number of Slave Interfaces : 1
- Number of Master Interfaces : 2

<img src="pictures/vivado_io_psinterconnect.png" alt="Configuring the PS Interconnect AXI Switch" style="width: 768px;"/>

### Concat (`plIrqConcat`)

Configure the Concat block named `plIrqConcat`.

You should match the settings shown in the picture below:

- Number of Ports: 1

<img src="pictures/vivado_io_irq_concat.png" alt="Configuring the IRQ Concat Block" style="width: 768px;"/>

### Concat (`pmodConcat`)

Next, configure the Concat block named `pmodConcat`.

You should match the settings shown in the picture below:

- Number of Ports: 8
- In0 to In3:
    - Manual
    - Width: 1

<img src="pictures/vivado_io_pmod_concat.png" alt="Configuring the PMOD Concat Block" style="width: 768px;"/>

### Concat (`uartRxSlice`)

Finally, configure the Slice block, named `uartRxSlice`.

You should match the settings shown in the picture below:

- Din Width: 8
- Din From: 1
- Din Down To: 1
- Dout Width: 1

<img src="pictures/vivado_io_slice.png" alt="Configuring the Slice Block" style="width: 768px;"/>


### Const (`xlConst_0`)

Finally, configure the Slice block, named `xlConst_0`.

You should match the settings shown in the picture below:

- Const Width: 8
- Const Val: 0xFB

<img src="pictures/vivado_io_slice.png" alt="Configuring the Const Block" style="width: 768px;"/>


## Wiring: 

For wiring, follow the directions below. You can change the visible layers (Clocks, GPIO, Interfaces, etc) by clicking the cog button in the upper right hand corner. 

### Clocks 

Connect all clocks to the `FCLK_CLK0` output of the ARM Processing System Block (processing_system_7_0) as shown below:

<img src="pictures/vivado_io_bd_clocks.png" alt="Clocks in the Block Diagram" style="width: 768px;"/>
    
### Resets

Using the block diagram editor, we will connect the available resets from the Reset Controllers. The Reset Controllers improve timing and ensure AXI-Standard resets. In general `interconnect_aresetn` connects to any `ARESETN` port on an AXI Interconnect, and `peripheral_aresetn` connects to any peripheral (like the UART Controller, or HLS core). `peripheral_aresetn` must drive the reset port of the peripheral **AND** the corresponding reset port of AXI interconnect.

The `userReset` controller is driven by a GPIO output from the ARM Processing System. This allows the user to reset the logic in the PL.

- **userReset**
    - Connect the `FCLK0_RESET_N` of `processing_system_7_0` to `aux_reset` of `userReset`
    - Expand `GPIO_0` (as in the number zero) of `processing_system_7_0` and connect `GPIO_O` (as in the alphabetical letter O) to `ext_reset` of `userReset`
        - This is shown in the IRQ & Other figure
    - Connect `peripheral_aresetn` of `userReset` to: 
        - `ap_rst_n` of `ctrlLoop`
        - `s_axi_aresetn` of `plIntrController`
        - `M00_ARESETN`, `S00_ARESETN`, and `M01_ARESETN` of `psIntereconnect`
        - `M00_ARESETN`, `S00_ARESETN`, of `hlsIntereconnect`
    - Connect `interconnect_aresetn` of `userReset` to: 
        - `ARESETN` of `psIntereconnect`
        - `ARESETN` of `hlsIntereconnect`
        
<img src="pictures/vivado_io_bd_resets.png" alt="Resets in the Block Diagram" style="width: 768px;"/>

### Interrupts & Other: 

We use interrupts to notify the ARM Processing System about events in the programmable logic. Wire the Interrupt Outputs as shown in the following picture. 

- Connect `pmodjA_data_in` to `Din0` of `uartRxSlice`
- Connect `Dout` of`uartRxSlice` to `rx` of `hlsUartLite`
- Connect `tx` of `hlsUartLite` to `in2` of `pmodConcat `
- Connect `dout` of `pmodConcat` to `pmodjA_data_out` 
- Connect `dout` of `xlConstant_0` to `pmodjA_tri_out`
- Connect `pb_in` to `buttons_V` of `ctrlLoop`
- Connect `leds_V` of `ctrlLoop` to `led`
- Connect `interrupt` of `ctrlLoop` to `In0` of `plIrqConcat`
- Connect `dout` of `plIrqConcat` to `intr` of `plIntrController`
- Connect `irq` of `plIntrController` to `IRQ_F2P` of `processing_system_0`
- Connect `GPIO_0` of `processing_system_0` to `ext_reset_in` of `userReset`

<img src="pictures/vivado_io_bd_other.png" alt="IRQs and Other wires in the Block Diagram" style="width: 768px;"/>

### AXI Memory Mapped Interfaces

AXI Memory Mapped interfaces are an addressed bus standard. Connect the AXI interfaces as shown in the picture below: 

- Connect `M_AXI_GP0` of `processing_system_7_0` to `S00_AXI` of `psInterconnect`
    - Connect `M00_AXI` of `psInterconnect` to `s_axi_CTRL` of `ctrlLoop`
    - Connect `M01_AXI` of `psInterconnect` to `s_axi` of `plIntrController`
- Connect `m_axi_IOMEM` of `ctrlLoop` to `S00_AXI` of `hlsInterconnect`
- Connect `M00_AXI` of `hlsInterconnect` to `s_axi` of `hlsUartLite`

<img src="pictures/vivado_io_bd_interfaces.png" alt="AXI Memory-Mapped Interfaces in the Block Diagram" style="width: 768px;"/>

Once these are connected, you will need to assign the address map. Click on the Address Editor tab and copy the settings from below: 

<img src="pictures/vivado_io_bd_memmap.png" alt="AXI Address Map in the Block Diagram" style="width: 768px;"/>

The address shown above are significant. On ZYNQ, the AXI port M_AXI_GP0 from the ARM Processing System to the FPGA Programmable Logic is mapped to the physical address range `0x4000_0000` to `0x8000_0000`. The `ctrlLoop` and `plInterruptContoller` are all mapped into this address space in contiguous 4K blocks.

You can find more information about the ZYNQ address map [here](https://www.xilinx.com/support/documentation/user_guides/ug585-Zynq-7000-TRM.pdf)

Once these addresses have been assigned, they will automagically be discovered in PYNQ from the `.tcl` file. 

You will also need to assign the address for the UART Controller in the HLS Core's memory space as shwon in the picture. 


## Checking your Progress
Once you have completed the wiring, you can validate your block design by selecting **Tools** -> **Validate Design** from the dropdown menu. 

If your design passes, then you can proceed! 

Save before proceeding to the next step.

## Exporting your Block Design

As the final step of this tutorial, we will export and compile your block design. 

Export your block design by clicking **File ** -> **Export...** -> **Export Block Design**. You will be presented with the following window: 

<img src="pictures/vivado_io_bd_export.png" alt="Exporting the Block Diagram" style="width: 512px;"/>

Remove the highlighted text and **OVERWRITE** the existing `~/PYNQ-HLS/tutorial/pynqhls/io/io.tcl` file.

When finished, close Vivado


## Compiling your Block Design

When Vivado has closed, use the makefile to rebuild the `io.bit` bitstream using the following commands: 

```bash
    cd ~/PYNQ-HLS/tutorial/pynqhls/io/
    make
```

This process can take up to 15 minutes. If your compilation completes without errors, proceeed to the next notebook, **[Using an HLS core in PYNQ](4-Using an HLS Core in PYNQ.ipynb)**.
